In [8]:
import pandas as pd
import pyodbc

DB = {'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
      'database': 'Practicum_4'}

# Constructing the connection string with Windows Authentication
conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

# Establishing the connection
export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor